In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2


--2023-07-05 05:33:39--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  15.4MB/s    in 6.6s    

2023-07-05 05:33:46 (9.21 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import dlib
import numpy as np
from PIL import Image
import os

In [ ]:
def helper(img_path, mask):
  image = cv2.imread(img_path)
  result = cv2.bitwise_and(image, mask)
  return result


In [ ]:
directory = "/content/drive/MyDrive/Clubbed_Images_parent/Clubbed images"
save_to = "/content/drive/MyDrive/Filtered_Human_Images/"


for filename in os.listdir(directory):
    club_directory = os.path.join(directory, filename)
    # print(club_directory)
    for filename2 in os.listdir(club_directory):

      if "8" in filename2.split("-"):
        name1 = os.path.join(club_directory, filename2)
      elif "32" in filename2.split("-"):
        name2 = os.path.join(club_directory, filename2)
      else:
        # Load the image
        image = cv2.imread(os.path.join(club_directory, filename2))
        # print(image.shape)
        # Convert the image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Initialize the dlib face detector and facial landmarks predictor
        detector = dlib.get_frontal_face_detector()
        # Download the .dat file from the internet.
        predictor = dlib.shape_predictor('/content/drive/MyDrive/Bank_dataset/shape_predictor_194_face_landmarks.dat')  # Replace with the path to your shape predictor model

        # Detect faces in the grayscale image
        faces = detector(gray)

        # Iterate over the detected faces and extract the facial region
        for face in faces:
          # Predict facial landmarks for the current face
          landmarks = predictor(gray, face)

          # Create a numpy array of the facial landmarks' coordinates
          landmarks_array = np.zeros((194, 2), dtype=int)
          for i in range(194):
            if i==0 or i==134:
              landmarks_array[i] = (landmarks.part(i).x, landmarks.part(i).y-125)
            else:
              landmarks_array[i] = (landmarks.part(i).x, landmarks.part(i).y)

          # Define the forehead region based on the facial landmarks
          # Define the region of interest (ROI) as the convex hull of the facial landmarks
          hull = cv2.convexHull(landmarks_array)

          # Create a mask of the same size as the original image and fill it with white
          mask = np.zeros_like(image)
          cv2.fillConvexPoly(mask, hull, (255, 255, 255))

          # Apply the mask to the original image
          result = cv2.bitwise_and(image, mask)


        # Display the result
        # cv2_imshow('Original Image', image)
        cv2_imshow(image)
        cv2_imshow(result)

        cv2.imwrite(save_to + filename2, result)

    cv2.imwrite(save_to + name1.split("/")[-1], helper(name1, mask))
    cv2.imwrite(save_to + name2.split("/")[-1], helper(name2, mask))
